In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import re
import copy
from scipy.signal import butter, filtfilt
import math

##### Gathering data

In [ ]:
mmmp_dir_pwp = './PWP'
fma_pwp = glob(os.path.join(mmmp_dir_pwp, '**/FMA*_P[0-9]*.tsv'), recursive=True) 

In [ ]:
tug_dir_pwp = r"\TUG\PWP"
tug_data_pwp = glob(os.path.join(tug_dir_pwp, '**/TUG02_P[0-9]*.tsv'), recursive=True)

In [ ]:
NAN_special = -999999
def check_file(file):
    # Load the motion capture data into a Pandas DataFrame
    df = pd.read_csv(file,sep = '\t',skiprows=11)
    df = df.fillna(NAN_special)
    df.columns = df.columns.str.strip() #eliminating unecesary spaces in column titles
    
    marker_list = ['RWRA X', 'RWRA Y', 'RWRA Z','LWRA X', 'LWRA Y', 'LWRA Z',
                   'RSHO X', 'RSHO Y', 'RSHO Z', 'LSHO X', 'LSHO Y','LSHO Z',
                  'LHEE X', 'LHEE Y', 'LHEE Z','RHEE X', 'RHEE Y', 'RHEE Z'
                   ,'LTOE X', 'LTOE Y', 'LTOE Z','RTOE X', 'RTOE Y', 'RTOE Z',
                   'STRN X', 'STRN Y', 'STRN Z',
                   'LASI X', 'LASI Y', 'LASI Z', 'RASI X', 'RASI Y', 'RASI Z',
                  'LELB X', 'LELB Y', 'LELB Z', 'RELB X', 'RELB Y', 'RELB Z']
    flag = False # this falg will help us identify the files that are incomplete later 
    for col_name in marker_list:
        if col_name not in df.columns:
            flag = True
            # print('File: {} , does not have marker {}'.format(file, col_name))

##### Function to read files

In [ ]:
NAN_special = -999999

def read_walking_file(file):
    # Load the motion capture data into a Pandas DataFrame
    df = pd.read_csv(file,sep = '\t',skiprows=11)
    df = df.fillna(NAN_special)
    df.columns = df.columns.str.strip() #eliminating unecesary spaces in column titles
    
    time = df.loc[:, "Time"].to_numpy()
    
#     subject_id, walk_phase = subject_id_walk_phase.split('_')[0],subject_id_walk_phase.split('_')[1]

    marker_list = ['RWRA X', 'RWRA Y', 'RWRA Z', 'LWRA X', 'LWRA Y', 'LWRA Z',
                   'RSHO X', 'RSHO Y', 'RSHO Z', 'LSHO X', 'LSHO Y', 'LSHO Z',
                   'LHEE X', 'LHEE Y', 'LHEE Z', 'RHEE X', 'RHEE Y', 'RHEE Z',
                   'LTOE X', 'LTOE Y', 'LTOE Z', 'RTOE X', 'RTOE Y', 'RTOE Z',
                   'STRN X', 'STRN Y', 'STRN Z',
                   'LASI X', 'LASI Y', 'LASI Z', 'RASI X', 'RASI Y', 'RASI Z',
                   'LELB X', 'LELB Y', 'LELB Z', 'RELB X', 'RELB Y', 'RELB Z',
                   'LANK X', 'LANK Y', 'LANK Z', 'RANK X', 'RANK Y', 'RANK Z']

    # Create empty arrays for each marker
    RSHO = np.zeros((len(time), 3))
    LSHO = np.zeros((len(time), 3))
    RWRA = np.zeros((len(time), 3))
    LWRA = np.zeros((len(time), 3))
    LHEE = np.zeros((len(time), 3))
    RHEE = np.zeros((len(time), 3))
    LTOE = np.zeros((len(time), 3))
    RTOE = np.zeros((len(time), 3))
    STRN = np.zeros((len(time), 3))
    LASI = np.zeros((len(time), 3))
    RASI = np.zeros((len(time), 3))
    LELB = np.zeros((len(time), 3))
    RELB = np.zeros((len(time), 3))
    LANK = np.zeros((len(time), 3))
    RANK = np.zeros((len(time), 3))

    flag = False # this flAg will help us identify the files that are incomplete later 
    missing_markers = [] # this is a list to collect the missing markers in the file

    for col_name in marker_list:
        if col_name in df.columns:

            if col_name.startswith('RSHO'):
                RSHO = df.loc[:, ['RSHO X', 'RSHO Y', 'RSHO Z']].to_numpy()
            elif col_name.startswith('LSHO'):
                LSHO = df.loc[:, ['LSHO X', 'LSHO Y', 'LSHO Z']].to_numpy()
            elif col_name.startswith('RWRA'):
                RWRA = df.loc[:, ['RWRA X', 'RWRA Y', 'RWRA Z']].to_numpy()
            elif col_name.startswith('LWRA'):
                LWRA = df.loc[:, ['LWRA X', 'LWRA Y', 'LWRA Z']].to_numpy()
            elif col_name.startswith('LHEE'):
                LHEE = df.loc[:, ['LHEE X', 'LHEE Y', 'LHEE Z']].to_numpy()
            elif col_name.startswith('RHEE'):
                RHEE = df.loc[:, ['RHEE X', 'RHEE Y', 'RHEE Z']].to_numpy()
            elif col_name.startswith('LTOE'):
                LTOE = df.loc[:, ['LTOE X', 'LTOE Y', 'LTOE Z']].to_numpy()
            elif col_name.startswith('RTOE'):
                RTOE = df.loc[:, ['RTOE X', 'RTOE Y', 'RTOE Z']].to_numpy()
            elif col_name.startswith('STRN'):
                STRN = df.loc[:, ['STRN X', 'STRN Y', 'STRN Z']].to_numpy()
            elif col_name.startswith('LASI'):
                LASI = df.loc[:, ['LASI X', 'LASI Y', 'LASI Z']].to_numpy()
            elif col_name.startswith('RASI'):
                RASI = df.loc[:, ['RASI X', 'RASI Y', 'RASI Z']].to_numpy()
            elif col_name.startswith('LELB'):
                LELB = df.loc[:, ['LELB X', 'LELB Y', 'LELB Z']].to_numpy()
            elif col_name.startswith('RELB'):
                RELB = df.loc[:, ['RELB X', 'RELB Y', 'RELB Z']].to_numpy()
            elif col_name.startswith('LANK'):
                LANK = df.loc[:, ['LANK X', 'LANK Y', 'LANK Z']].to_numpy()
            elif col_name.startswith('RANK'):
                RANK = df.loc[:, ['RANK X', 'RANK Y', 'RANK Z']].to_numpy()
        else:
            flag = True
            missing_markers.append(col_name)
    print('File: {} , does not have markers {}'.format(file, missing_markers))
    
    return missing_markers, time, RSHO, LSHO, RWRA, LWRA, LHEE, RHEE, LTOE, RTOE, STRN, LASI, RASI, LELB, RELB, LANK, RANK#, subject_id, walk_phase



In [ ]:
missing_markers, t, RSHO, LSHO, RWRA, LWRA, LHEE, RHEE, LTOE, RTOE, STRN, LASI, RASI, LELB, RELB, LANK, RANK = read_walking_file(fma_pwp[-4])

### Walking speed (Cowie et al., 2012)
    Calculated with the pelvis midpoint in the walking direction.
    Freeze Like Episode: period in which velocity drops below 10% of baseline. (what is baseline in our case?)

In [ ]:
 def smooth_and_clean_velocity(position_data, time_data, cutoff_freq=5, fs=100):
    """
    Smooths position data and calculates clean velocity
    
    Args:
        position_data: Position data in meters
        time_data: Time stamps
        cutoff_freq: Cutoff frequency for low-pass filter (Hz)
        fs: Sampling frequency (Hz)
    """
    # First smooth the position data
    b, a = butter(4, cutoff_freq/(fs/2), btype='low')
    position_smoothed = filtfilt(b, a, position_data)
    
    # Calculate velocity using central difference
    velocity = np.zeros(len(position_smoothed))
    velocity[1:-1] = (position_smoothed[2:] - position_smoothed[:-2]) / (time_data[2:] - time_data[:-2])
    
    # Handle endpoints
    velocity[0] = (position_smoothed[1] - position_smoothed[0]) / (time_data[1] - time_data[0])
    velocity[-1] = (position_smoothed[-1] - position_smoothed[-2]) / (time_data[-1] - time_data[-2])
    
    # Apply physiological constraints
    max_walking_speed = 2.7  # maximum realistic walking speed in m/s
    velocity = np.clip(velocity, -max_walking_speed, max_walking_speed)
    
    return velocity   

def walking_velocity_1d(LASI,RASI,time,missing_markers_list):   
 
    # Check if more than 40% of the markers in y are missing
    # if np.sum(LASI[:, 1] == NAN_special) / len(LASI[:, 1]) > 0.4 or np.sum(RASI[:, 1] == NAN_special) / len(RASI[:, 1]) > 0.4:
    #     print('More than 40% of the markers in y are missing, assigning walking velocity as 0')
    #     return 0, np.zeros_like(LASI[:, 1])

    if 'RASI Y' and 'LASI Y' in missing_markers_list:
        print('RASI and LASI are missing, assigning walking velocity as 0')
        return 0, np.zeros_like(time) # we return a null value in case hip markers are missing

    elif 'RASI Y' in missing_markers_list:
        print('RASI is missing, assigning pelvis coordinates only with LASI')
        y_position_pelvis = LASI[:, 1]

    elif 'LASI Y' in missing_markers_list:
        print('LASI is missing, assigning pelvis coordinates only with RASI')
        y_position_pelvis = RASI[:, 1]

    else:
        pelvis = (LASI + RASI) / 2 # calculate pelvis as midpoint between LASI and RASI
        y_position_pelvis = pelvis[:, 1]    
    
    # Convert position to meters
    y_position_pelvis = y_position_pelvis / 1000  # mm to meters
    
    # find the peak index in the y-axis / extreme point in y postition to determine return 
    peak_index = np.argmax(y_position_pelvis)
    
    # set the threshold as peak value minus 2*std(pelvis in y)
    threshold = y_position_pelvis[peak_index] - 2 * np.std(y_position_pelvis)


    # Search the left tail starting index
    
    left_tail_index = peak_index #starting at max point 
    while left_tail_index > 0 and y_position_pelvis[left_tail_index] > threshold: 
        left_tail_index -= 1 #move one position backwards until close to threshold

    # Search right tail starting index
    right_tail_index = peak_index #starting at max point 
    while right_tail_index < len(y_position_pelvis) - 1 and y_position_pelvis[right_tail_index] > threshold:
        right_tail_index += 1 #move one position forward until close to threshold

    # Define left and right segments of the pelvis y-position
    
    left_segment_pos_pelvis = y_position_pelvis[left_tail_index:peak_index]
    # print('left tail index:',left_tail_index)
    # # print(left_segment_pos_pelvis)
    right_segment_pos_pelvis = y_position_pelvis[peak_index:right_tail_index]

    # Calculate velocity for each segment, cutting the time vector to match both segments
    
    if left_tail_index == 0:
        print('Departure phase missing on TUG file, defining baseline velocity as the mean of the return segment')
        pelvis_velocity_y = np.abs(smooth_and_clean_velocity(right_segment_pos_pelvis, time[peak_index:right_tail_index]))
        mean_walking_vel = np.mean(pelvis_velocity_y)
    
    elif right_tail_index == 0:
        print('Return phase missing on TUG file, defining baseline velocity as the mean of the departure segment')
        pelvis_velocity_y = smooth_and_clean_velocity(left_segment_pos_pelvis, time[left_tail_index:peak_index])
        mean_walking_vel = np.mean(pelvis_velocity_y)
    else:
        vel_left = smooth_and_clean_velocity(left_segment_pos_pelvis, time[left_tail_index:peak_index])
        vel_right = smooth_and_clean_velocity(right_segment_pos_pelvis, time[peak_index:right_tail_index])
        pelvis_velocity_y = np.concatenate((vel_left, np.abs(vel_right)))
        mean_walking_vel = np.mean(pelvis_velocity_y)
            
    # print(pelvis_velocity_y.shape)
    # plt.plot(pelvis_velocity_y, label='Pelvis Velocity')
    # # plt.plot(y_delta_pos_pelvis_left, label='Left Segment')
    # # plt.plot(y_delta_pos_pelvis_right, label='Right Segment')
    # plt.show()

    return mean_walking_vel, pelvis_velocity_y     #both in abs value because of change in direction during return 


mean_walking_vel,pelvis_velocity_y = walking_velocity_1d(LASI,RASI,t,missing_markers)
print(mean_walking_vel)
plt.figure(figsize=(6, 3))
plt.plot(pelvis_velocity_y, label='Smoothed Velocity', alpha=0.8)
plt.axhline(y=mean_walking_vel, color='r', linestyle='--', label='Mean Velocity')
plt.xlabel('Time points')
plt.ylabel('Velocity (m/s)')
plt.legend()
plt.grid(True)
plt.show()

## Baseline velocity from TUG

In [ ]:
def read_baseline_walking_file(file):
    # Load the motion capture data into a Pandas DataFrame
    df = pd.read_csv(file,sep = '\t',skiprows=11)
    df = df.fillna(NAN_special)
    df.columns = df.columns.str.strip() #eliminating unecesary spaces in column titles
    
    time = df.loc[:, "Time"].to_numpy()
    
#     subject_id, walk_phase = subject_id_walk_phase.split('_')[0],subject_id_walk_phase.split('_')[1]

    marker_list = ['LASI X', 'LASI Y', 'LASI Z', 'RASI X', 'RASI Y', 'RASI Z']

    # Create empty arrays for each marker
   
    LASI = np.zeros((len(time), 3))
    RASI = np.zeros((len(time), 3))
    
    flag = []
    missing_markers = [] 

    for col_name in marker_list:
        if col_name in df.columns:
            flag.append(False)
            if col_name.startswith('LASI'):
                LASI = df.loc[:, ['LASI X', 'LASI Y', 'LASI Z']].to_numpy()
            elif col_name.startswith('RASI'):
                RASI = df.loc[:, ['RASI X', 'RASI Y', 'RASI Z']].to_numpy()
        else:
            flag.append(True)
            missing_markers.append(col_name)
        
    print('File: {} , does not have markers {}'.format(file, missing_markers))

    return missing_markers, time, LASI, RASI

missing_markers, t, LASI, RASI = read_baseline_walking_file(tug_data_pwp[-2])

In [ ]:
def get_baseline_velocity_1d(bl_file):
    
    missing_markers_list, t, LASI, RASI = read_baseline_walking_file(bl_file)

    if 'RASI Y' and 'LASI Y' in missing_markers_list:
        print('RASI and LASI are missing, assigning baseline velocity as 0')
        return baseline_velocity == 0 # we return a null value in case hip markers are missing

    elif 'RASI Y' in missing_markers_list:
        print('RASI is missing, assigning pelvis coordinates only with LASI')
        y_position_pelvis = LASI[:, 1]

    elif 'LASI Y' in missing_markers_list:
        print('LASI is missing, assigning pelvis coordinates only with RASI')
        y_position_pelvis = RASI[:, 1]

    else:
        pelvis = (LASI + RASI) / 2 # calculate pelvis as midpoint between LASI and RASI
        y_position_pelvis = pelvis[:, 1]    
    
     # Convert position to meters
    y_position_pelvis = y_position_pelvis / 1000  # mm to meters
    
    # find the peak index in the y-axis / extreme point in y postition to determine return 
    peak_index = np.argmax(y_position_pelvis)
    
    # set the threshold as peak value minus 2*std(pelvis in y)
    threshold = y_position_pelvis[peak_index] - 2 * np.std(y_position_pelvis)

    # there might not be left or right tail so we need to check if 
    # the y pelvis curve corresponds to a gaussian-like curve or not
        
    # Search the left tail starting index
    
    left_tail_index = peak_index #starting at max point 
    while left_tail_index > 0 and y_position_pelvis[left_tail_index] > threshold: 
        left_tail_index -= 1 #move one position backwards until close to threshold

    # Search right tail starting index
    right_tail_index = peak_index #starting at max point 
    while right_tail_index < len(y_position_pelvis) - 1 and y_position_pelvis[right_tail_index] > threshold:
        right_tail_index += 1 #move one position forward until close to threshold

    # Define left and right segments of the pelvis y-position
    
    left_segment_pos_pelvis = y_position_pelvis[left_tail_index:peak_index]
    # print('left tail index:',left_tail_index)
    # # print(left_segment_pos_pelvis)
    right_segment_pos_pelvis = y_position_pelvis[peak_index:right_tail_index]

    # Calculate the time and position differentials for velocity
    time_diff = np.diff(t)

    # Calculate velocity for each segment, cutting the time vector to match both segments
    
    if left_tail_index == 0:
        print('Departure phase missing on TUG file, defining baseline velocity as the mean of the return segment')
        pelvis_velocity_y = np.abs(smooth_and_clean_velocity(right_segment_pos_pelvis, t[peak_index:right_tail_index]))
        return np.mean(pelvis_velocity_y)
    
    elif right_tail_index == 0:
        print('Return phase missing on TUG file, defining baseline velocity as the mean of the departure segment')
        pelvis_velocity_y = np.abs(smooth_and_clean_velocity(left_segment_velocity, t[peak_index:right_tail_index]))
        return np.mean(pelvis_velocity_y)
    else:
        vel_left = smooth_and_clean_velocity(left_segment_pos_pelvis, t[left_tail_index:peak_index])
        vel_right = smooth_and_clean_velocity(right_segment_pos_pelvis, t[peak_index:right_tail_index])
        pelvis_velocity_y = np.concatenate((vel_left, np.abs(vel_right)))
        baseline_velocity = np.mean(pelvis_velocity_y)

        # plt.plot(vel_left, label='Left Segment')
        # plt.plot(np.abs(vel_right), label='Right Segment')
        
           # Plotting the pelvis position and detected thresholds
    #     plt.figure(figsize=(12, 6))
    #     plt.plot(t, y_position_pelvis, label='Pelvis Y Position', color='blue')
    #     plt.axvline(x=t[peak_index], color='orange', linestyle='--', label='Peak')
    #     plt.axvline(x=t[left_tail_index], color='green', linestyle='--', label='Left Tail')
    #     plt.axvline(x=t[right_tail_index], color='red', linestyle='--', label='Right Tail')
        
    #     plt.xlabel('Time (s)')
    #     plt.ylabel('Pelvis Y Position (mm)')
    #     plt.title('Pelvis Y Position with Left and Right Tail Detection')
    #     plt.legend()
    #     plt.grid(True)
    #     plt.show()
    return baseline_velocity

baseline_velocity = get_baseline_velocity_1d(tug_data_pwp[-2])
print(baseline_velocity)

In [ ]:
plt.axhline(baseline_velocity,label='Baseline velocity',color='red')
plt.axhline(baseline_velocity*0.1,label='Threshold',color='green')

plt.plot(pelvis_velocity_y,label='Velocity in y-axis')
plt.legend(loc='upper right')
plt.ylabel('Velocity m/s')
plt.xlabel('Time (s)')
# plt.ylim(-0.3,1)


# Freeze-like Events

Based on the calculation criteria of: Cowie et al., 2012 `"... defined an FLE as a period in which velocity dropped below 10% of baseline." `

This code calculates the duration of every FLE, and provides a counts how many FLE are in every trajectory.
`A FLE is counted if it has a minimum duration of 0.5 seconds`

In [ ]:
def fle(baseline_vel,pelvis_velocity_vector,time_vector):
    baseline_threshold = 0.1 * baseline_vel 
    
    # Identify FLE points (True where velocity is below 10% of baseline)
    is_fle = pelvis_velocity_vector < baseline_threshold

    # Find the start and end indices of each FLE
    fle_durations = []
    fle_count = 0
    in_fle = False  # track if we're inside a FLE

    for i in range(1, len(is_fle)):
        if is_fle[i] and not in_fle:  # Start of a new FLE
            fle_start = i
            in_fle = True
        elif not is_fle[i] and in_fle:  # End of the current FLE
            fle_end = i
            fle_duration = time_vector[fle_end] - time_vector[fle_start]
            if fle_duration >= 0.5: # Threshold: of FLE is long enough = at least 0.5 seconds (to avoid noisy velocity points) - Cowie : no threshold but in graph is as low as 0.1 s
                fle_durations.append(fle_duration)
                fle_count += 1
            else: 
                 pass
            in_fle = False


    # Output results with handling for empty fle_durations
#     print(f"Number of FLEs: {fle_count}")
    if fle_durations:
        mean_fle_duration = np.mean(fle_durations)
#         print(f"Durations of each FLE (in seconds): {fle_durations}")
#         print(f"Mean FLE time (in seconds): {np.mean(fle_durations)}")
    else:
        mean_fle_duration = 0
#         print("No freeze-like events detected.")
#     print(fle_durations)
            
    return fle_count, mean_fle_duration



count_fle, mean_fle_time = fle(baseline_velocity,pelvis_velocity_y,t)
print(count_fle, mean_fle_time)


## Storing FLE data in dataframe

In [ ]:
#Create the dataframes to store the gait parameters
df_fle_fmap = pd.DataFrame(columns=['Subject ID','Group','TrialFMA', 
                                              'Mean_Walking_velocity[m/s]',
                                              'TUG_baseline_velocity[m/s]',
                                              'total_FLE','Mean_FLE_duration[s]'
                                             ] )


In [ ]:
full_data = [fma_control,fma_pwp]
# subject_ids = [ids_control,ids_pwp]

group_label = ['Control','PwP']
# tug_data_pwp = tug_data_pwp

for i,group in enumerate(full_data):
    for j,file in enumerate(group):
        # Obtain labels
        g_l = group_label[i]
        
        # Extract subject ID
        if g_l == 'Control':
            subject_id = re.search(r'_C(\d+)\.tsv$', file).group(1)
            subject_id = 'C'+subject_id
            color = 'blue'
        else: 
            subject_id = re.search(r'_P(\d+)\.tsv$', file).group(1)
            subject_id = 'P'+subject_id

        
        # Extract trial ID
        # Extract trial ID with error handling
        match = re.search(r'FMA0?(\d+)_', file)  # '0?' makes the zero optional
        if match:
            trial = match.group(1)
        else:
            # Handle missing trial ID (e.g., raise error, log warning, set default)
            raise ValueError(f"Trial ID not found in filename: {file}")

        
        # Read file
        missing_markers, t, RSHO, LSHO, RWRA, LWRA, LHEE, RHEE, LTOE, RTOE, STRN, LASI, RASI, LELB, RELB, LANK, RANK = read_walking_file(file)
       
        # # Check if there are markers missing
        # if flag[0]==True:
        #     print(f'Subject {subject_id}, does not have marker {flag[1]}')
            
        # # Check for 'departure' suffixes
        # for suffix in ['01', '03', '05']:
        #     if f'_{suffix}_' in file:
        #         code = suffix
        #         break  # Stop once a match is found
        
        # # Check for 'return' suffixes if no 'departure' was found
        # else:
        #     for suffix in ['02', '04', '06']:
        #         if f'_{suffix}_' in file:
        #             code = suffix
        #             break          
            
        #------------------------- 
        #------------------------- WALKING VELOCITY
        
        mean_walking_vel,pelvis_velocity_y = walking_velocity_1d(LASI,RASI,t,missing_markers)
        
            # mean_walking_vel,pelvis_velocity_y = np.abs(mean_walking_vel), np.abs(pelvis_velocity_y)
        
        
        #------------------------- 
        
        #------------------------- FLE and BASELINE VELOCITY FROM TUG TASK (ONLY FOR PWP)
        if g_l == 'PwP':
            # Search the Parciticpant's corresponding TUG file in tug_data_pwp that matches its subject_id
            corresponding_file = [f for f in tug_data_pwp 
                                  if subject_id == f.split("\\")[-1].split("_")[-1].split(".")[0]]
            
            
            # Check if a corresponding file was found
            if corresponding_file:
                print(corresponding_file)
                # Calculate baseline velocty, save it in baseline_vel 
                baseline_file = corresponding_file[0]
                # print(subject_id)
                baseline_vel = get_baseline_velocity_1d(baseline_file)
                
                # Call the FLE function
                
                count_fle, mean_fle_time = fle(baseline_vel,pelvis_velocity_y,t)
                
#                 print(f"Found corresponding file for {subject_id}: {corresponding_file[0]}")
                
            else:
                print(f"No TUG02 file found for {subject_id}")
        else:
            # Null values for Controls: 
            baseline_vel = 0
            count_fle = 0
            mean_fle_time = 0
                    
        
        #------------------------- 
        
        #------------------------- SAVING DATA IN DF
        
        new_line = {'Subject ID':subject_id,'Group':g_l,'TrialFMA':trial,
                   'Mean_Walking_velocity[m/s]': mean_walking_vel,
                    'TUG_baseline_velocity[m/s]':baseline_vel,
                    'total_FLE':count_fle,
                    'Mean_FLE_duration[s]':mean_fle_time,
                        }
        
        df_fle_fmap.loc[len(df_fle_fmap)] = new_line

In [ ]:
df_fle_fmap

# Merge with covariates for ANCOVA